## Exporting agriculture patents

In [2]:
from flashtext import KeywordProcessor
import time
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

### Time of execution
start = time.time()

terms = ['agriculture','agricultural','agritech','agtech']# list(catalogue.keywords)
print('terms to use: {}'.format(len(terms)))
data_laws = pd.read_csv('datasets/laws_and_policies_06072020.csv')
data_laws['type'] = 'laws_and_policies'
data_cases = pd.read_csv('datasets/litigation_cases_06072020.csv')
data_cases['type'] = 'litigation_cases'

print('total laws: {}'.format(len(data_laws)))
print('total cases: {}'.format(len(data_cases)))

def extract(vec, dictionary, info=False):
    matrix = []
    for line in vec:
        matrix.append(dictionary.extract_keywords(str(line), span_info=info))
    return matrix

#proccess
data = data_laws[['Title','Description','type']]\
    .rename(columns={'Description':'Summary'})\
    .append(data_cases[['Title','Summary','type']])
dictionary = KeywordProcessor()
dictionary.add_keywords_from_list(terms)
extracted = extract(data.Summary, dictionary)
row = [list(set(i)) if len(i)>0 else '' for i in extracted]
data['matches'] = [str(i).replace('[', '').replace(']', '') for i in row]
data['count_matches'] = [len(i) for i in extracted]
data['count_unique_matches'] = [len(set(i)) for i in extracted]

#export data
export = data[data['count_matches'] > 0].sort_values('count_matches', ascending=False)
export.to_excel('datasets/agri_laws_and_cases.xlsx')
print('agri_laws_and_cases.xlsx')
end = time.time()
print('Elapsed time: {}'.format(time.strftime("%H:%M:%S", time.gmtime(end - start))))

terms to use: 4
total laws: 1884
total cases: 375
agri_laws_and_cases.xlsx
Elapsed time: 00:00:00
